In [1]:
import torch
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import pickle

import numpy as np
import os
import pandas as pd
from utils.vmf_batch import vMF

from models import SeqEncoder, SeqDecoder, Seq2SeqDataSet, Seq2Seq_VAE, PoolingClassifier, init_weights
from utils.training_utils import create_Seq2SeqDataset, train, evaluate

## plotting ###

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
SEED = 17
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
path = './data/%s/iterator/%s/%s.pkl'
part = 'm_labels'
folder = 'M1_exc_data'

with open(path%(folder, part, 'train_iterator'), 'rb') as f:
    train_iterator = pickle.load(f)
    
with open(path%(folder, part, 'val_iterator'), 'rb') as f:
    val_iterator = pickle.load(f)    
    
    


In [5]:
src_data, trg_data, seq_len, indices, labels = list(train_iterator)[0]
bs, n_walks, walk_length, output_dim = src_data.shape

N_train = len(train_iterator.sampler.indices)
N_val = len(val_iterator.sampler.indices)

torch.cuda.empty_cache()

### load model

In [6]:
emb_dim = 32
latent_dim = 32
output_dim = 3
n_walks = 256
NUM_LAYERS = 2
MASKING_ELEMENT = train_iterator.dataset.masking_el
dpout = .1
kap = 500
pool = 'max'
lr = 0.01

NUM_CLASSES = 3

enc = SeqEncoder(output_dim, emb_dim, emb_dim, NUM_LAYERS, dpout)
dec = SeqDecoder(output_dim, emb_dim, emb_dim, NUM_LAYERS, dpout)
dist = vMF(latent_dim, kappa=kap)

model = Seq2Seq_VAE(enc, dec, dist, device).to(device)
classifier = PoolingClassifier(latent_dim, NUM_CLASSES, n_walks,dpout,pooling=pool).to(device)

KLD: 45.709938049316406


In [7]:
def calculate_loss(x, reconstructed_x, ignore_el=MASKING_ELEMENT):
    # reconstruction loss
    # x = [trg len, batch size * n walks, output dim]

    seq_len , bs, output_dim = x.shape
    mask = x[:,:,0] != ignore_el
    RCL = 0
    for d in range(output_dim):
        RCL += mse_loss(reconstructed_x[:,:,d][mask], x[:,:,d][mask])
    RCL /= output_dim
    
    return RCL

In [11]:

N_EPOCHS= 200
save_path_model= './models/M1_exc/m_label/finetuned_vae_k500_frac%.1f_best_run%i.pt'
save_path_losses = './models/M1_exc/m_label/finetuned_losses_frac%.1f_run%i.npy'
save_path_elapsed_time = './models/M1_exc/m_label/finetuned_scaled_elapsed_time_frac%.1f_run%i.npy'

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
for frac in [1., .9, .5, .1, 0.]:
    runs = range(1,4)
        
    for run in runs:
        
        #optimizer
        optimizer = optim.Adam(list(model.parameters()) + list(classifier.parameters()), lr=lr)
        
        if os.path.exists(save_path_model%(frac,run)):
            state_dict = torch.load(save_path_model%(frac,run))
            
            # load model
            model.load_state_dict(state_dict['model_state_dict'])
            
            # overwrite optimizer if the model had been trained already
            optimizer.load_state_dict(state_dict['optimizer_state_dict'])
            classifier.load_state_dict(state_dict['classifier_state_dict'])
            losses = np.load(save_path_losses%(frac, run))
            elapsed_time = np.load(save_path_elapsed_time%(frac, run))
            
            last_epoch = state_dict['epoch']
            training = list(losses[:last_epoch,:2])
            validation = list(losses[:last_epoch,2:])
            elapsed_time = elapsed_time[:last_epoch]
            elapsed_time = np.hstack((elapsed_time, np.zeros((N_EPOCHS))))
            best_test_loss = losses[:,2].min()
      
        else:
            # load pre-trained model
            state_dict = torch.load('./models/5_populations/emb32_hid32_lat32_dp0.1_k500_max_frac1.0_scaled_sum_run1_best.pt')
            # the first run was the best
            model.load_state_dict(state_dict['model_state_dict'])
            classifier.apply(init_weights)
            best_test_loss = np.infty

            losses = np.load('./models/5_populations/losses_emb32_hid32_lat32_dp0.1_k500_max_frac1.0_scaled_sum_1.npy')
            elapsed_time = np.load('./models/Farrow/scratch/soma_centered/elapsed_time_frac0.0_scaled_run%i.npy'%run)
            last_epoch = 0
            elapsed_time = np.zeros((N_EPOCHS))
            training = list(losses[:,:2])
            validation = list(losses[:,2:])
            best_test_loss = np.infty
        
        
        cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=-100)
        mse_loss = torch.nn.MSELoss(reduction='sum')
       

        for e in range(last_epoch, N_EPOCHS):
            start.record()
            train_loss, train_class_loss = train(model, classifier, train_iterator, optimizer, 
                                               calculate_loss,cross_entropy_loss, clip=1, norm_p=None,
                                                 class_fraction=frac)
            val_loss, val_class_loss = evaluate(model,classifier, val_iterator,
                                                 calculate_loss, cross_entropy_loss, norm_p=None)


            train_loss /= N_train
            train_class_loss /= N_train
            val_loss /= N_val
            val_class_loss /=N_val
            
            end.record()

            # Waits for everything to finish running
            torch.cuda.synchronize()
            elapsed_time[e+last_epoch] = start.elapsed_time(end) # milliseconds

            training += [[train_loss,train_class_loss]]
            validation += [[val_loss, val_class_loss]]
            print(f'Epoch {e}, Train Loss: {train_loss:.2f}, Val Loss: {val_loss:.2f}, Time elapsed [s]: {elapsed_time[e]/1000:.2f}')


            if e % 50 == 0 and e > 0:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']/2

            if best_test_loss > val_loss:
                best_test_loss = val_loss
                torch.save({'epoch': e + last_epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'classifier_state_dict': classifier.state_dict()
                               },save_path_model%(frac, run))

                validation_ = np.array(validation)
                training_ = np.array(training)
                # [:,0] = training loss, [:,1] = training classification loss 
                # [:,2] validation loss, [:,3] validation classification loss
                losses = np.hstack((training_, validation_))
                np.save(save_path_losses%(frac, run),losses)
                np.save(save_path_elapsed_time%(frac,run),elapsed_time)
        validation = np.array(validation)
        training = np.array(training)
        losses = np.hstack((training, validation))
        np.save(save_path_losses%(frac, run), losses)
        np.save(save_path_elapsed_time%(frac,run),elapsed_time)

Epoch 0, Train Loss: 6046.33, Val Loss: 22843.59, Time elapsed [s]: 10.65
Epoch 1, Train Loss: 5535.66, Val Loss: 2084.81, Time elapsed [s]: 10.50
Epoch 2, Train Loss: 922.68, Val Loss: 1408.33, Time elapsed [s]: 10.42
Epoch 3, Train Loss: 697.09, Val Loss: 967.32, Time elapsed [s]: 10.50
Epoch 4, Train Loss: 456.53, Val Loss: 1394.99, Time elapsed [s]: 10.24
Epoch 5, Train Loss: 518.68, Val Loss: 1075.30, Time elapsed [s]: 11.20
Epoch 6, Train Loss: 473.28, Val Loss: 952.10, Time elapsed [s]: 13.64
Epoch 7, Train Loss: 406.11, Val Loss: 706.11, Time elapsed [s]: 10.54
Epoch 8, Train Loss: 317.33, Val Loss: 679.59, Time elapsed [s]: 10.76
Epoch 9, Train Loss: 304.06, Val Loss: 625.63, Time elapsed [s]: 10.76
Epoch 10, Train Loss: 288.59, Val Loss: 568.72, Time elapsed [s]: 10.52
Epoch 11, Train Loss: 275.45, Val Loss: 581.18, Time elapsed [s]: 10.44
Epoch 12, Train Loss: 261.48, Val Loss: 564.63, Time elapsed [s]: 10.25
Epoch 13, Train Loss: 249.90, Val Loss: 498.56, Time elapsed [s]: 

Epoch 114, Train Loss: 91.51, Val Loss: 251.73, Time elapsed [s]: 10.53
Epoch 115, Train Loss: 91.81, Val Loss: 240.62, Time elapsed [s]: 11.26
Epoch 116, Train Loss: 93.08, Val Loss: 252.47, Time elapsed [s]: 10.76
Epoch 117, Train Loss: 86.01, Val Loss: 244.64, Time elapsed [s]: 11.12
Epoch 118, Train Loss: 87.06, Val Loss: 234.88, Time elapsed [s]: 10.72
Epoch 119, Train Loss: 89.45, Val Loss: 226.10, Time elapsed [s]: 12.33
Epoch 120, Train Loss: 84.58, Val Loss: 235.10, Time elapsed [s]: 11.77
Epoch 121, Train Loss: 83.69, Val Loss: 220.38, Time elapsed [s]: 11.32
Epoch 122, Train Loss: 85.33, Val Loss: 221.47, Time elapsed [s]: 12.57
Epoch 123, Train Loss: 91.58, Val Loss: 227.19, Time elapsed [s]: 11.80
Epoch 124, Train Loss: 87.91, Val Loss: 230.69, Time elapsed [s]: 11.41
Epoch 125, Train Loss: 81.68, Val Loss: 221.91, Time elapsed [s]: 12.07
Epoch 126, Train Loss: 84.58, Val Loss: 237.74, Time elapsed [s]: 10.77
Epoch 127, Train Loss: 86.30, Val Loss: 260.84, Time elapsed [s]

Epoch 28, Train Loss: 187.96, Val Loss: 523.25, Time elapsed [s]: 11.08
Epoch 29, Train Loss: 188.68, Val Loss: 485.85, Time elapsed [s]: 10.91
Epoch 30, Train Loss: 195.92, Val Loss: 473.05, Time elapsed [s]: 10.99
Epoch 31, Train Loss: 186.20, Val Loss: 571.95, Time elapsed [s]: 11.20
Epoch 32, Train Loss: 192.26, Val Loss: 426.46, Time elapsed [s]: 10.82
Epoch 33, Train Loss: 176.65, Val Loss: 559.46, Time elapsed [s]: 11.18
Epoch 34, Train Loss: 197.03, Val Loss: 401.29, Time elapsed [s]: 11.39
Epoch 35, Train Loss: 189.93, Val Loss: 497.48, Time elapsed [s]: 11.52
Epoch 36, Train Loss: 168.00, Val Loss: 517.97, Time elapsed [s]: 10.50
Epoch 37, Train Loss: 204.90, Val Loss: 410.30, Time elapsed [s]: 11.27
Epoch 38, Train Loss: 165.78, Val Loss: 385.41, Time elapsed [s]: 11.47
Epoch 39, Train Loss: 160.49, Val Loss: 365.67, Time elapsed [s]: 11.54
Epoch 40, Train Loss: 163.28, Val Loss: 405.66, Time elapsed [s]: 11.51
Epoch 41, Train Loss: 157.79, Val Loss: 549.31, Time elapsed [s]

Epoch 142, Train Loss: 88.32, Val Loss: 236.79, Time elapsed [s]: 11.05
Epoch 143, Train Loss: 91.07, Val Loss: 236.31, Time elapsed [s]: 12.66
Epoch 144, Train Loss: 87.20, Val Loss: 228.83, Time elapsed [s]: 10.68
Epoch 145, Train Loss: 87.81, Val Loss: 237.02, Time elapsed [s]: 10.68
Epoch 146, Train Loss: 92.04, Val Loss: 220.75, Time elapsed [s]: 14.11
Epoch 147, Train Loss: 83.89, Val Loss: 229.48, Time elapsed [s]: 14.14
Epoch 148, Train Loss: 94.19, Val Loss: 222.61, Time elapsed [s]: 10.77
Epoch 149, Train Loss: 83.53, Val Loss: 229.19, Time elapsed [s]: 10.81
Epoch 150, Train Loss: 83.65, Val Loss: 243.12, Time elapsed [s]: 10.57
Epoch 151, Train Loss: 94.41, Val Loss: 219.63, Time elapsed [s]: 10.95
Epoch 152, Train Loss: 81.81, Val Loss: 281.61, Time elapsed [s]: 11.11
Epoch 153, Train Loss: 90.55, Val Loss: 249.43, Time elapsed [s]: 12.08
Epoch 154, Train Loss: 89.99, Val Loss: 216.97, Time elapsed [s]: 11.16
Epoch 155, Train Loss: 78.07, Val Loss: 231.51, Time elapsed [s]

Epoch 56, Train Loss: 138.73, Val Loss: 305.63, Time elapsed [s]: 10.78
Epoch 57, Train Loss: 133.14, Val Loss: 385.37, Time elapsed [s]: 10.91
Epoch 58, Train Loss: 136.11, Val Loss: 305.84, Time elapsed [s]: 10.71
Epoch 59, Train Loss: 126.42, Val Loss: 321.53, Time elapsed [s]: 10.84
Epoch 60, Train Loss: 126.88, Val Loss: 301.67, Time elapsed [s]: 11.19
Epoch 61, Train Loss: 134.81, Val Loss: 326.27, Time elapsed [s]: 10.86
Epoch 62, Train Loss: 126.09, Val Loss: 294.25, Time elapsed [s]: 10.74
Epoch 63, Train Loss: 119.11, Val Loss: 380.42, Time elapsed [s]: 14.30
Epoch 64, Train Loss: 152.29, Val Loss: 374.92, Time elapsed [s]: 14.11
Epoch 65, Train Loss: 139.30, Val Loss: 411.02, Time elapsed [s]: 10.72
Epoch 66, Train Loss: 134.76, Val Loss: 307.56, Time elapsed [s]: 13.01
Epoch 67, Train Loss: 123.36, Val Loss: 339.37, Time elapsed [s]: 14.52
Epoch 68, Train Loss: 130.25, Val Loss: 305.35, Time elapsed [s]: 13.50
Epoch 69, Train Loss: 119.99, Val Loss: 371.44, Time elapsed [s]

...

Epoch 87, Train Loss: 67.39, Val Loss: 321.64, Time elapsed [s]: 10.71
Epoch 88, Train Loss: 66.14, Val Loss: 301.79, Time elapsed [s]: 10.70
Epoch 89, Train Loss: 65.59, Val Loss: 313.80, Time elapsed [s]: 10.74
Epoch 90, Train Loss: 66.71, Val Loss: 297.46, Time elapsed [s]: 11.14
Epoch 91, Train Loss: 64.39, Val Loss: 289.71, Time elapsed [s]: 11.11
Epoch 92, Train Loss: 62.26, Val Loss: 284.89, Time elapsed [s]: 10.97
Epoch 93, Train Loss: 68.71, Val Loss: 277.97, Time elapsed [s]: 10.45
Epoch 94, Train Loss: 66.04, Val Loss: 301.35, Time elapsed [s]: 10.61
Epoch 95, Train Loss: 70.75, Val Loss: 302.19, Time elapsed [s]: 11.25
Epoch 96, Train Loss: 65.77, Val Loss: 277.20, Time elapsed [s]: 10.69
Epoch 97, Train Loss: 65.30, Val Loss: 335.02, Time elapsed [s]: 11.23
Epoch 98, Train Loss: 69.58, Val Loss: 290.01, Time elapsed [s]: 10.98
Epoch 99, Train Loss: 65.09, Val Loss: 299.98, Time elapsed [s]: 10.99
Epoch 100, Train Loss: 64.83, Val Loss: 281.03, Time elapsed [s]: 12.40
Epoch

Epoch 1, Train Loss: 5460.70, Val Loss: 2034.23, Time elapsed [s]: 10.70
Epoch 2, Train Loss: 821.89, Val Loss: 1367.23, Time elapsed [s]: 11.52
Epoch 3, Train Loss: 579.91, Val Loss: 971.82, Time elapsed [s]: 11.27
Epoch 4, Train Loss: 353.95, Val Loss: 975.11, Time elapsed [s]: 11.65
Epoch 5, Train Loss: 357.49, Val Loss: 967.15, Time elapsed [s]: 10.68
Epoch 6, Train Loss: 310.20, Val Loss: 801.14, Time elapsed [s]: 11.80
Epoch 7, Train Loss: 256.56, Val Loss: 659.91, Time elapsed [s]: 10.95
Epoch 8, Train Loss: 228.45, Val Loss: 836.97, Time elapsed [s]: 11.09
Epoch 9, Train Loss: 235.90, Val Loss: 772.37, Time elapsed [s]: 10.79
Epoch 10, Train Loss: 281.74, Val Loss: 600.90, Time elapsed [s]: 11.74
Epoch 11, Train Loss: 205.68, Val Loss: 722.95, Time elapsed [s]: 11.06
Epoch 12, Train Loss: 198.65, Val Loss: 680.62, Time elapsed [s]: 11.61
Epoch 13, Train Loss: 198.65, Val Loss: 641.68, Time elapsed [s]: 11.11
Epoch 14, Train Loss: 171.93, Val Loss: 633.42, Time elapsed [s]: 11.0

Epoch 116, Train Loss: 61.39, Val Loss: 254.78, Time elapsed [s]: 10.41
Epoch 117, Train Loss: 53.39, Val Loss: 260.57, Time elapsed [s]: 12.18
Epoch 118, Train Loss: 50.99, Val Loss: 260.83, Time elapsed [s]: 11.75
Epoch 119, Train Loss: 53.86, Val Loss: 343.50, Time elapsed [s]: 12.30
Epoch 120, Train Loss: 66.65, Val Loss: 333.02, Time elapsed [s]: 12.31
Epoch 121, Train Loss: 58.25, Val Loss: 258.04, Time elapsed [s]: 10.80
Epoch 122, Train Loss: 60.50, Val Loss: 290.13, Time elapsed [s]: 10.93
Epoch 123, Train Loss: 63.55, Val Loss: 259.78, Time elapsed [s]: 10.95
Epoch 124, Train Loss: 58.52, Val Loss: 261.01, Time elapsed [s]: 11.23
Epoch 125, Train Loss: 55.54, Val Loss: 297.05, Time elapsed [s]: 14.15
Epoch 126, Train Loss: 65.98, Val Loss: 274.12, Time elapsed [s]: 11.77
Epoch 127, Train Loss: 59.31, Val Loss: 294.06, Time elapsed [s]: 10.47
Epoch 128, Train Loss: 62.97, Val Loss: 285.20, Time elapsed [s]: 10.33
Epoch 129, Train Loss: 52.08, Val Loss: 271.73, Time elapsed [s]

Epoch 30, Train Loss: 128.52, Val Loss: 477.80, Time elapsed [s]: 11.68
Epoch 31, Train Loss: 118.98, Val Loss: 445.74, Time elapsed [s]: 11.23
Epoch 32, Train Loss: 116.69, Val Loss: 433.42, Time elapsed [s]: 10.55
Epoch 33, Train Loss: 108.17, Val Loss: 411.56, Time elapsed [s]: 10.85
Epoch 34, Train Loss: 103.95, Val Loss: 431.29, Time elapsed [s]: 10.89
Epoch 35, Train Loss: 96.15, Val Loss: 391.71, Time elapsed [s]: 10.91
Epoch 36, Train Loss: 96.97, Val Loss: 401.20, Time elapsed [s]: 11.27
Epoch 37, Train Loss: 102.51, Val Loss: 507.52, Time elapsed [s]: 13.05
Epoch 38, Train Loss: 133.32, Val Loss: 470.73, Time elapsed [s]: 10.92
Epoch 39, Train Loss: 139.20, Val Loss: 565.76, Time elapsed [s]: 10.53
Epoch 40, Train Loss: 124.55, Val Loss: 399.51, Time elapsed [s]: 10.63
Epoch 41, Train Loss: 110.20, Val Loss: 414.78, Time elapsed [s]: 10.71
Epoch 42, Train Loss: 116.57, Val Loss: 499.88, Time elapsed [s]: 11.38
Epoch 43, Train Loss: 115.07, Val Loss: 451.99, Time elapsed [s]: 

Epoch 145, Train Loss: 56.60, Val Loss: 250.03, Time elapsed [s]: 12.27
Epoch 146, Train Loss: 52.14, Val Loss: 263.79, Time elapsed [s]: 11.56
Epoch 147, Train Loss: 54.57, Val Loss: 259.07, Time elapsed [s]: 10.59
Epoch 148, Train Loss: 50.45, Val Loss: 260.30, Time elapsed [s]: 12.04
Epoch 149, Train Loss: 53.98, Val Loss: 277.80, Time elapsed [s]: 11.08
Epoch 150, Train Loss: 47.85, Val Loss: 244.69, Time elapsed [s]: 11.46
Epoch 151, Train Loss: 49.87, Val Loss: 267.65, Time elapsed [s]: 10.32
Epoch 152, Train Loss: 52.28, Val Loss: 268.23, Time elapsed [s]: 10.33
Epoch 153, Train Loss: 49.58, Val Loss: 257.64, Time elapsed [s]: 10.59
Epoch 154, Train Loss: 61.15, Val Loss: 271.26, Time elapsed [s]: 10.40
Epoch 155, Train Loss: 62.34, Val Loss: 249.83, Time elapsed [s]: 10.64
Epoch 156, Train Loss: 47.98, Val Loss: 269.37, Time elapsed [s]: 10.74
Epoch 157, Train Loss: 49.90, Val Loss: 237.17, Time elapsed [s]: 10.60
Epoch 158, Train Loss: 52.61, Val Loss: 251.39, Time elapsed [s]